In [3]:
# ! pip install meilisearch
# ! pip install pandas

In [3]:
import meilisearch
import json
import pandas as pd

In [ ]:
client = meilisearch.Client('http://localhost:7700')
index = client.index('brick_classes_and_definitions') #select the index which you want to match againt. This can be brick_classes, hto_tags, etc.

# Codes

In [5]:
codes = pd.read_excel('./inputs/codes/codes_nl_en_pp.xlsx')
# codes.drop(columns=codes.columns[0], axis=1, inplace=True)
codes[codes.columns] = codes.apply(lambda x: x.str.strip())
# codes['Code Description (EN)'] = codes['Code Description (EN)'].str.replace(' ', '_')
codes.head()

,Code,Code Description (NL),Code Description (EN),Code Brick Class
0,AKM,Absorptiekoelmachine,Absorption chiller,Absorption_Chiller
1,TGC,Toegangscontrole,Access control,Access_Control_Equipment
2,CR-,Cardreader,Card reader,Access_Reader
3,FA-,Flowalarm,Flow alarm,Air_Flow_Alarm
4,QC,NaN,Air quality,Air_Quality_Sensor


### Map the 'Description Code' column

In [6]:

# recm = codes['Code Description (EN)'].map(client.index('brick_classes_and_definitions').search)

recm = codes['Code Description (EN)'].apply(lambda text: index.search(text, {'limit': 3}))

recm

0      {'hits': [{'id': 6, 'brick_class': 'https://br...
1      {'hits': [{'id': 12, 'brick_class': 'https://b...
2      {'hits': [], 'query': 'Card reader', 'processi...
3      {'hits': [{'id': 17, 'brick_class': 'https://b...
4      {'hits': [{'id': 672, 'brick_class': 'https://...
                             ...                        
112    {'hits': [{'id': 2447, 'brick_class': 'https:/...
113    {'hits': [], 'query': 'Key safe', 'processingT...
114    {'hits': [], 'query': 'Tourniquet', 'processin...
115    {'hits': [{'id': 2483, 'brick_class': 'https:/...
116    {'hits': [{'id': 1083, 'brick_class': 'https:/...
Name: Code Description (EN), Length: 117, dtype: object

In [7]:
pd.DataFrame(recm)
recm.head()
recm[1]

{'hits': [{'id': 12,
   'brick_class': 'https://brickschema.org/schema/Brick#Access_Control_Equipment'},
  {'id': 1631,
   'brick_class': 'https://brickschema.org/schema/Brick#Underfloor_Air_Plenum',
   'definition': 'An open space between a structural concrete slab and the underside of a raised access floor system that connects to an air handling unit to receive conditioned and/or ventilating air before delivery to the room(s)'},
  {'id': 709,
   'brick_class': 'https://brickschema.org/schema/Brick#Hazardous_Materials_Storage',
   'definition': 'A storage space set aside (usually with restricted access) for the storage of materials that can be hazardous to living beings or the environment'}],
 'query': 'Access control',
 'processingTimeMs': 7,
 'limit': 3,
 'offset': 0,
 'estimatedTotalHits': 4}

In [8]:
out1 = list(map(lambda s: s['hits'], recm))
out2 = list(map(lambda s: ",".join(list(map(lambda i: i['brick_class'], s))), out1))
print(f"Length is :{len(out2)}, {out2[0]}")

Length is :117, https://brickschema.org/schema/Brick#Chiller,https://brickschema.org/schema/Brick#Absorption_Chiller


In [9]:
codes['Recommendations'] = list(out2)
codes['Recommendations'] = codes['Recommendations'].str.replace('https://brickschema.org/schema/Brick#', '')
codes.head(10)

C:\Users\lasit\AppData\Local\Temp\ipykernel_19340\2316422128.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  codes['Recommendations'] = codes['Recommendations'].str.replace('https://brickschema.org/schema/Brick#', '')


,Code,Code Description (NL),Code Description (EN),Code Brick Class,Recommendations
0,AKM,Absorptiekoelmachine,Absorption chiller,Absorption_Chiller,"Chiller,Absorption_Chiller"
1,TGC,Toegangscontrole,Access control,Access_Control_Equipment,"Access_Control_Equipment,Underfloor_Air_Plenum..."
2,CR-,Cardreader,Card reader,Access_Reader,
3,FA-,Flowalarm,Flow alarm,Air_Flow_Alarm,"Air_Flow_Loss_Alarm,Flow,Flow_Setpoint"
4,QC,NaN,Air quality,Air_Quality_Sensor,"Air_Quality_Sensor,Air_Quality,Air"
5,QT-,Lucht kwaliteit,Air quality,Air_Quality_Sensor,"Air_Quality_Sensor,Air_Quality,Air"
6,ST-,Storingsmelding,Fault alarm,Alarm,"Fault_Reset_Command,Last_Fault_Code_Status,Hum..."
7,KE-,Ketelinstallatie,Boiler installation,Boiler,"Equipment,Boiler,Fire_Zone"
8,VB-,Booster ventilator,booster fan,Booster_Fan,Booster_Fan
9,RK-,Regelkast,"Control box, Switch box",Breaker_Panel,"Motor_Control_Center,Thermostat,Overridden_Off..."


In [10]:
codes[['Recom 1', 'Recom 2', 'Recom 3']] = codes['Recommendations'].str.split(',', expand=True)

codes.drop('Recommendations', axis=1, inplace=True)

codes.head(15)

,Code,Code Description (NL),Code Description (EN),Code Brick Class,Recom 1,Recom 2,Recom 3
0,AKM,Absorptiekoelmachine,Absorption chiller,Absorption_Chiller,Chiller,Absorption_Chiller,None
1,TGC,Toegangscontrole,Access control,Access_Control_Equipment,Access_Control_Equipment,Underfloor_Air_Plenum,Hazardous_Materials_Storage
2,CR-,Cardreader,Card reader,Access_Reader,,None,None
3,FA-,Flowalarm,Flow alarm,Air_Flow_Alarm,Air_Flow_Loss_Alarm,Flow,Flow_Setpoint
4,QC,NaN,Air quality,Air_Quality_Sensor,Air_Quality_Sensor,Air_Quality,Air
5,QT-,Lucht kwaliteit,Air quality,Air_Quality_Sensor,Air_Quality_Sensor,Air_Quality,Air
6,ST-,Storingsmelding,Fault alarm,Alarm,Fault_Reset_Command,Last_Fault_Code_Status,Humidifier_Fault_Status
7,KE-,Ketelinstallatie,Boiler installation,Boiler,Equipment,Boiler,Fire_Zone
8,VB-,Booster ventilator,booster fan,Booster_Fan,Booster_Fan,None,None
9,RK-,Regelkast,"Control box, Switch box",Breaker_Panel,Motor_Control_Center,Thermostat,Overridden_Off_Status


In [11]:
codes.to_excel('./meilisearch/outputs/codes_meili.xlsx')

In [12]:
mask = codes[['Recom 1', 'Recom 2', 'Recom 3']].isin(codes['Code Brick Class']).any(axis=1)

# filter the dataframe by the mask
filtered_df = codes[mask]

# display the filtered dataframe
print(f'Total: {len(codes)}, Matched: {len(filtered_df)}, Percentage {len(filtered_df)/len(codes)*100}')

Total: 117, Matched: 35, Percentage 29.914529914529915


# Last Letter

In [13]:
last_letter = pd.read_excel('./inputs/codes/last_letter_pp.xlsx')
# last_letter.drop(columns=last_letter.columns[0], axis=1, inplace=True)
last_letter.head()

recm = last_letter['Last_Letter Description (EN)'].apply(lambda text: index.search(text, {'limit': 3}))

out1 = list(map(lambda s: s['hits'], recm))
out2 = list(map(lambda s: ",".join(list(map(lambda i: i['brick_class'], s))), out1))

last_letter['Recommendations'] = list(out2)
last_letter['Recommendations'] = last_letter['Recommendations'].str.replace('https://brickschema.org/schema/Brick#', '')

last_letter[['Recom 1', 'Recom 2', 'Recom 3']] = last_letter['Recommendations'].str.split(',', expand=True)

last_letter.drop('Recommendations', axis=1, inplace=True)

last_letter.head()

C:\Users\lasit\AppData\Local\Temp\ipykernel_19340\559789516.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  last_letter['Recommendations'] = last_letter['Recommendations'].str.replace('https://brickschema.org/schema/Brick#', '')


,Last_Letter,Last_Letter Description (NL),Last_Letter Description (EN),Last_Letter Brick Class,Recom 1,Recom 2,Recom 3
0,A--,Alarm,Alarm,Alarm,Alarm,Alarm_Delay_Parameter,Water_Alarm
1,B--,Bedrijfsmelding,Operation status,Status,Overridden_Status,Status,Chilled_Water_System_Enable_Command
2,D--,Dicht,Closed,Open_Close_Status,Condensing_Natural_Gas_Boiler,Natural_Gas_Boiler,Electric_Boiler
3,HS--,Hoofdschakelaar,Main switch,Switchgear,Switchgear,Bus_Riser,Disconnect_Switch
4,L--,Lokaal bediend,Locally operated,Status,,None,None


In [14]:
last_letter.to_excel('./meilisearch/outputs/last_letter_meili.xlsx')

In [15]:
mask = last_letter[['Recom 1', 'Recom 2', 'Recom 3']].isin(last_letter['Last_Letter Brick  Class']).any(axis=1)

# filter the dataframe by the mask
filtered_df = last_letter[mask]

# display the filtered dataframe
print(f'Total: {len(last_letter)}, Matched: {len(filtered_df)}, Percentage {len(filtered_df)/len(last_letter)*100}')

Total: 34, Matched: 12, Percentage 35.294117647058826


# Numbers

In [16]:
number_code = pd.read_excel('./inputs/codes/numbers_nl_en_pp.xlsx')
# number_code.drop(columns=codes.columns[0], axis=1, inplace=True)
number_code.head()

# recm = number_code['Description Num'].map(client.index('brick_classes_and_definitions').search)
# recm.head()

recm = number_code['Num Description (EN)'].apply(lambda text: index.search(text, {'limit': 3}))

out1 = list(map(lambda s: s['hits'], recm))
out2 = list(map(lambda s: ",".join(list(map(lambda i: i['brick_class'], s))), out1))

print(f"Length is :{len(out2)}, {out2[0]}")

number_code['Recommendations'] = list(out2)

number_code['Recommendations'] = number_code['Recommendations'].str.replace('https://brickschema.org/schema/Brick#', '')

number_code[['Recom 1', 'Recom 2', 'Recom 3']] = number_code['Recommendations'].str.split(',', expand=True)

number_code.drop('Recommendations', axis=1, inplace=True)

number_code.head()

Length is :39, https://brickschema.org/schema/Brick#Boiler,https://brickschema.org/schema/Brick#Fire_Zone,https://brickschema.org/schema/Brick#Hot_Water_Differential_Temperature_Sensor


C:\Users\lasit\AppData\Local\Temp\ipykernel_19340\2342444412.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  number_code['Recommendations'] = number_code['Recommendations'].str.replace('https://brickschema.org/schema/Brick#', '')


,Num,Num Description (NL),Num Description (EN),Num Brick,Recom 1,Recom 2,Recom 3
0,1,Ketel 1,Boiler 1,Boiler,Boiler,Fire_Zone,Hot_Water_Differential_Temperature_Sensor
1,2,Ketel 2,Boiler 2,Boiler,Boiler,Fire_Zone,Hot_Water_Differential_Temperature_Sensor
2,11,Transportsysteem ketel 1,Transport system boiler 1,Hot_Water_Loop,Chilled_Water_Loop,Water_Loop,Hot_Water_Loop
3,12,Transportsysteem ketel 2,Transport system boiler 2,Hot_Water_Loop,Chilled_Water_Loop,Water_Loop,Hot_Water_Loop
4,21,Warm water groep 1,Hot water group 1,Hot_Water_System,Hot_Water,Disable_Hot_Water_System_Outside_Air_Temperatu...,Enable_Hot_Water_System_Outside_Air_Temperatur...


In [17]:
number_code.to_excel('./meilisearch/outputs/numbers_meili.xlsx')

In [18]:
mask = number_code[['Recom 1', 'Recom 2', 'Recom 3']].isin(number_code['Num Brick']).any(axis=1)

# filter the dataframe by the mask
filtered_df = number_code[mask]

# display the filtered dataframe
print(f'Total: {len(number_code)}, Matched: {len(filtered_df)}, Percentage {len(filtered_df)/len(number_code)*100}')


Total: 39, Matched: 12, Percentage 30.76923076923077
